In [1]:
import json
!pip install -Uq requests
import requests
!mkdir -p data

# Using an API

Learning outcomes

- difference between API & webscraping
- what JSON is (and why it's like a Python `dict`)
- how to properly handle files in Python
- what a REST API is
- how to use the `requests` library

## API versus web-scraping

**Both are ways to sample data from the internet**

API
- structured
- provided as a service (you are talking to a server via a REST API)
- limited data / rate limits / paid / require auth (sometimes)
- most will give back JSON (maybe XML or CSV)

Web scraping
- less structure
- parsing HTML meant for your browser

Neither is better than the other

- API developer can limit what data is accessible through the API
- API developer can not maintain the API
- website page can change HTML structure
- website page can have dynamic (Javascript) content that requires execution (usually done by the browser) before the correct HTML is available

Much of the work in using an API is figuring out how to properly construct URL's for `GET` requests
- requires looking at their documentation (& ideally a Python example!)

## Where to find APIs

- [ProgrammableWeb](https://www.programmableweb.com/apis/directory) - a collection of available API's
- For the *Developer* or *For Developers* documentation on your favourite website
- [public-apis/public-apis](https://github.com/public-apis/public-apis)

## Using API's

Most API's require authentication

- so they API developer knows who you are
- can charge you
- can limit access
- commonly via key or OAuth (both of which may be free)

All the API's we use here are unauthenticated - this is to avoid the time of you all signing up

If your app requries authentication, it's usually done by passing in your credentials into the request (as a header)

```python
response = requests.get(url, auth=auth)
```

## JSON strings

JSON (JavaScript Object Notation) is a:
- lightweight data-interchange format (text)
- easy for humans to read and write 
- easy for machines to parse and generate
- based on key, value pairs

You can think of the Python `dict` as JSON like:

In [10]:
data = {'name': 'alan-turing'}
data

{'name': 'alan-turing'}

But true JSON is just a string (only text).  We can use `json.dumps` from the standard library to turn the `dict` into a JSON string:

In [11]:
data = json.dumps(data)
data

'{"name": "alan-turing"}'

In [12]:
type(data)

str

We can then use `json.loads` to turn this string back into a `dict`:

In [13]:
json.loads(data)

{'name': 'alan-turing'}

## Opening, reading & writing to files

### Reading from a file

We open files using the Python `open` builtin function, followed by a `read`:

In [15]:
open('./readme.md', 'r').read()[:100]

'# Python\n\nWhy Python?\n\n- fast to write - suitable for research\n- allows quick iteration over ideas\n-'

(Note the `./path` - the `.` refers to the current working directory.  It's not straightforward to know where this is - is it where the notebook is, or where the notebook server is running?

If we wanted to read the file as separate lines, we could use `readlines()` (note we would still need to manually strip off the `\n` characters later)

(Note `/n` is used as a newline indicator in text files - you never see it because your editor interprets it as a line break :)

In [16]:
open('./readme.md', 'r').readlines()[:8]

['# Python\n',
 '\n',
 'Why Python?\n',
 '\n',
 '- fast to write - suitable for research\n',
 '- allows quick iteration over ideas\n',
 '- can be put into production\n',
 '- massive ecosystem\n']

## Using `open`

`open(path, mode)`

Common values for the mode:
- `r` read
- `rb` read binary
- `w+` write (`+` to create file if it doesn't exist)
- `a` append

Note there are options for both reading & writing - we actually use `open` for both reading & writing.

We open a file using the Python builtin `open`, which is then followed by either a read or write stage

- open the file
- read the file OR write to the file

Notice that the file is read in as a single string, with the newline character `\n` separating lines

- this is how all text files are structured
- your editor does the line splitting for you

### Writing to a file (without context management)

We can write to a new file using the same `open` builtin

- open the file
- write to the file

In [19]:
open('./data/output.data', 'w').write('We make this file to show how not to do it\n')

43

Note that we can do the same file write by explicitly assiging the file object to a variable (note the `a` to append)

In [20]:
fi = open('./data/output.data', 'a')
fi.write('We make this file slightly differently to show how not to do it\n')

64

The issue with the code above is that we aren't closing the file - we can fix this by intentionally closing the file.  

One way to do this is to use `.close()` when we are done:

In [21]:
fi = open('./data/output.data', 'a')
fi.write('This time we close the file manually\n')
fi.close()

This requires us to remember to close (also an additional line).

### Reading files with context management

The Pythonic way of handling opening & closing of files is context management:

In [22]:
with open('./readme.md', 'r') as fi:
    data = fi.read()

### Writing to a file with context management

Now that we understand context management, we can save our `data` dict as JSON, using `json.dump` to write the dict to a file:

In [23]:
data = {'name': 'alan turing'}
with open('./data/output.json', 'w') as fi:
    json.dump(data, fi)

Let's check it worked by loading the file, here using `json.load` to load from the file object:

In [24]:
with open('./data/output.json', 'r') as fi:
    data = json.load(fi)
data

{'name': 'alan turing'}

## REST API's

[REST - Wiki](https://en.wikipedia.org/wiki/Representational_state_transfer)

REST is a set of constraints that allow **stateless communication of text data on the internet**

- REST = REpresentational State Transfer
- API = Application Programming Interface

REST
- communication of resources (located at URLs / URIs)
- requests for a resource are responded to with a text payload (HTML, JSON etc)
- these requests are made using HTTP (determines how messages are formatted, what actions (methods) can be taken)
- common HTTP methods are `GET` and `POST`

HTTP methods
- GET - retrieve information about the REST API resource
- POST - create a REST API resource
- PUT - update a REST API resource
- DELETE - delete a REST API resource or related component

RESTful APIs enable you to develop any kind of web application having all possible CRUD (create, retrieve, update, delete) operations

- can do anything we would want to do with a database

*Further reading*
- [Web Architecture 101](https://engineering.videoblocks.com/web-architecture-101-a3224e126947) for more detail on how the web works

## Example - sunrise API

Docs - https://sunrise-sunset.org/api

First we need to form the url
- use `?` to separate the API server name from the parameters for our request
- use `&` to separate the parameters from each other
- use `+` instead of space in the parameter

In [25]:
res = requests.get("https://api.sunrise-sunet.org/json?lat=36.7201600&lng=-4.4203400")
data = res.json()
data

{'results': {'sunrise': '6:06:20 AM',
  'sunset': '6:13:45 PM',
  'solar_noon': '12:10:02 PM',
  'day_length': '12:07:25',
  'civil_twilight_begin': '5:40:36 AM',
  'civil_twilight_end': '6:39:28 PM',
  'nautical_twilight_begin': '5:10:28 AM',
  'nautical_twilight_end': '7:09:37 PM',
  'astronomical_twilight_begin': '4:39:55 AM',
  'astronomical_twilight_end': '7:40:10 PM'},
 'status': 'OK'}

This response is JSON - `requests.json()` turns it into a `dict`:

In [26]:
type(data)

dict

It's common to have a top level heirarchy to dig through to get the data:

In [28]:
data.keys()

dict_keys(['results', 'status'])

Here the interesting stuff is in `results`:

In [29]:
data['results']

{'sunrise': '6:06:20 AM',
 'sunset': '6:13:45 PM',
 'solar_noon': '12:10:02 PM',
 'day_length': '12:07:25',
 'civil_twilight_begin': '5:40:36 AM',
 'civil_twilight_end': '6:39:28 PM',
 'nautical_twilight_begin': '5:10:28 AM',
 'nautical_twilight_end': '7:09:37 PM',
 'astronomical_twilight_begin': '4:39:55 AM',
 'astronomical_twilight_end': '7:40:10 PM'}

## Example - Chronicling America API

Docs - https://chroniclingamerica.loc.gov/about/api/ 

In [30]:
term = "germany"
fmt = "json"
url = f"https://chroniclingamerica.loc.gov/search/pages/results/?proxtext={term}&format={fmt}"
url

'https://chroniclingamerica.loc.gov/search/pages/results/?proxtext=germany&format=json'

We use the `requests` HTTP library to perform a `GET` request:

In [31]:
response = requests.get(url)

## HTTP response

What we recieved above is an *HTTP response**

[HTTP Response - Wiki](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Response_message)

The response message consists of the following:

- a status line which includes the status code and reason message (e.g., HTTP/1.1 200 OK, which indicates that the client's request succeeded)
- response header fields (e.g., Content-Type: text/html)
- an optional message body

## What can we do with this HTTP response in `requests`?

The Python builtin `dir` gives us all the attributes & methods of a Python object.

This also includes all the `__` dunder (literally double-under) methods) - which we filter out using a list comprehension.

In [32]:
[o for o in dir(response) if '__' not in o]

['_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

We can get the HTTP status code (used to communicate things like everything OK (200), stop making requests etc - see [List of HTTP status codes - Wiki](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)):

In [33]:
response.status_code

200

The HTTP response headers:

In [34]:
response.headers

{'Date': 'Tue, 22 Sep 2020 08:11:44 GMT', 'Content-Type': 'application/json', 'Content-Length': '7886', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d7eefc151a32400cc13a351cc494bec3c1600762304; expires=Thu, 22-Oct-20 08:11:44 GMT; path=/; domain=.loc.gov; HttpOnly; SameSite=Lax', 'Last-Modified': 'Tue, 22 Sep 2020 06:04:40 GMT', 'Expires': 'Wed, 23 Sep 2020 06:04:40 GMT', 'Cache-Control': 's-maxage=86400, public, max-age=86400', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'X-requested-with', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Varnish': '858603556 838362604', 'Age': '96', 'Via': '1.1 varnish (Varnish/5.2)', 'CF-Cache-Status': 'HIT', 'Accept-Ranges': 'bytes', 'cf-request-id': '0556776eba0000dfc39319b200000001', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '5d6a8e91296cdfc3-FRA'}

And the response body:

In [35]:
response.text[:1000]

'{"totalItems": 2087178, "endIndex": 20, "startIndex": 1, "itemsPerPage": 20, "items": [{"sequence": 16, "county": ["San Francisco"], "edition": null, "frequency": "Daily", "id": "/lccn/sn85066387/1908-11-08/ed-1/seq-16/", "subject": ["California--San Francisco Bay Area.--fast--(OCoLC)fst01242397", "California--San Francisco.--fast--(OCoLC)fst01204481", "San Francisco (Calif.)--Newspapers.", "San Francisco Bay Area (Calif.)--Newspapers."], "city": ["San Francisco"], "date": "19081108", "title": "The San Francisco call. [volume]", "end_year": 1913, "note": ["\\"San Francisco\\" appears above, and later across, masthead ornament.", "Also issued online.", "Archived issues are available in digital format from the Library of Congress Chronicling America online collection.", "Issued with a joint ed. of the San Francisco chronicle and the San Francisco examiner on the day after the San Francisco earthquake, Apr. 19, 1906.", "Master negatives are available for duplication from:", "Publishers: 

In [37]:
import json
data = json.loads(response.text)

We can access the keys of the dictionary:

In [38]:
data.keys()

dict_keys(['totalItems', 'endIndex', 'startIndex', 'itemsPerPage', 'items'])

We can access the values using the square bracket indexing with a key:

In [39]:
data['totalItems']

2087178

While JSON is a simple text format, it can become complex due to

- nesting (JSON inside JSON)
- lists of JSON

An example is our `items`, which has been parsed as a Python `list`:

In [40]:
type(data['items'])

list

`items` is a list of dicts:

In [41]:
item = data['items'][0]
item.keys()

dict_keys(['sequence', 'county', 'edition', 'frequency', 'id', 'subject', 'city', 'date', 'title', 'end_year', 'note', 'state', 'section_label', 'type', 'place_of_publication', 'start_year', 'edition_label', 'publisher', 'language', 'alt_title', 'lccn', 'country', 'ocr_eng', 'batch', 'title_normal', 'url', 'place', 'page'])

We can iterate over both the keys and values as a pair using `items()`.

Below we use a check  a quick check that the value isn't too long before printing:

In [42]:
from collections.abc import Iterable

for k, v in item.items():
    if isinstance(v, Iterable) and len(v) < 100:
        print(f'{k}: {v}')

county: ['San Francisco']
frequency: Daily
id: /lccn/sn85066387/1908-11-08/ed-1/seq-16/
subject: ['California--San Francisco Bay Area.--fast--(OCoLC)fst01242397', 'California--San Francisco.--fast--(OCoLC)fst01204481', 'San Francisco (Calif.)--Newspapers.', 'San Francisco Bay Area (Calif.)--Newspapers.']
city: ['San Francisco']
date: 19081108
title: The San Francisco call. [volume]
note: ['"San Francisco" appears above, and later across, masthead ornament.', 'Also issued online.', 'Archived issues are available in digital format from the Library of Congress Chronicling America online collection.', 'Issued with a joint ed. of the San Francisco chronicle and the San Francisco examiner on the day after the San Francisco earthquake, Apr. 19, 1906.', 'Master negatives are available for duplication from:', 'Publishers: Charles M. Shortridge, <1896>; John D. Spreckles, <1899>.']
state: ['California']
section_label: 
type: page
place_of_publication: San Francisco [Calif.]
edition_label: 
publi

Let's finish this exercise by only taking articles that appear between two years, and save those to disk.

Normally you would apply this kind of filtering in the API request - we are going to filter in memory.

First we need a bit of data cleaning of our date, which is an integer representation of time (but is a `str`):

In [43]:
item['date']

'19081108'

Here use `strptime` to convert the integer into a proper datetime:
- ([Python's strftime directives](http://strftime.org/) is very useful!)

In [44]:
from datetime import datetime as dt
dt.strptime(item['date'], "%Y%m%d")

datetime.datetime(1908, 11, 8, 0, 0)

Now let's put this data cleaning & filtering into a pipeline:

In [45]:
term = "germany"
fmt = "json"
url = f"https://chroniclingamerica.loc.gov/search/pages/results/?proxtext={term}&format={fmt}"
res = requests.get(url)
data = res.json()
items = data['items']

start = 1900
extract = []
for item in items:
    item['date'] = dt.strptime(item['date'], "%Y%m%d")
    
    if item['date'].year > start:
        extract.append(item)
        
len(extract)

20

We have a list of dictionaries, which plays very nice with `pandas`:

In [46]:
!pip install -q pandas
import pandas as pd
df = pd.DataFrame(extract)
df.head(2)

,sequence,county,edition,frequency,id,subject,city,date,title,end_year,note,state,section_label,type,place_of_publication,start_year,edition_label,publisher,language,alt_title,lccn,country,ocr_eng,batch,title_normal,url,place,page
0,16,[San Francisco],None,Daily,/lccn/sn85066387/1908-11-08/ed-1/seq-16/,[California--San Francisco Bay Area.--fast--(O...,[San Francisco],1908-11-08,The San Francisco call. [volume],1913,"[""San Francisco"" appears above, and later acro...",[California],,page,San Francisco [Calif.],1895,,Charles M. Shortridge,[English],"[Call, Call-chronicle-examiner, Sunday call]",sn85066387,California,"JACK— he w.45 11111^^ Germany,\nI^ll/ riFSIIOV...",curiv_llano_ver01,san francisco call.,https://chroniclingamerica.loc.gov/lccn/sn8506...,[California--San Francisco--San Francisco],16
1,40,[None],None,Daily,/lccn/sn85038615/1905-10-08/ed-1/seq-40/,"[Richmond (Va.)--Newspapers., Virginia--Richmo...",[Richmond],1905-10-08,The times dispatch. [volume],1914,"[Also issued on microfilm from Bell & Howell, ...",[Virginia],MAGAZINE SECTION,page,"Richmond, Va.",1903,,Times-Dispatch Co.,[English],[],sn85038615,Virginia,njuvv/w*r\nHE GETS SQUARE!\nHOCH DER7 HOOLIGAN...,vi_berea_ver01,times dispatch.,https://chroniclingamerica.loc.gov/lccn/sn8503...,[Virginia--Richmond],


## Example - downloading images

We can also `requests` to download things other than text - such as images.

Below we do a requests and see we get back a binary string:

In [47]:
url = 'https://www.google.com/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png'
res = requests.get(url)
res.text[:100]

'�PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02 \x00\x00\x00�\x08\x06\x00\x00\x00�#W\x1b\x00\x004�IDATx\x01��\x03�%?\x1e��m۶}����(�Y:��n��z��m۶���w�$=�ח�\u05fb�^����N�g���|��_'

We can use context management to dump the contents of the binary string into a file:

In [48]:
with open('./data/google-logo.png', 'wb') as fi:
    fi.write(res.content)

We now have the Google logo locally (you may need to re-run this cell)

![](./data/google-logo.png)

## Exericse (group) - earthquake API 

Let's as a group write a program to get data from the USGS Earthquake Catalog - [documentation](https://earthquake.usgs.gov/fdsnws/event/1/#methods)

In [89]:
start = "2014-01-01"
end = "2014-01-02"
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start}&endtime={end}"

import os
os.makedirs('./data/earthquakes', exist_ok=True)
os.makedirs(f'./data/earthquakes/{start}', exist_ok=True)

response = requests.get(url)
data = response.json()
features = data['features']
for feature in features:
    properties = feature['properties']
  
    url = properties['url'].split('/')[-1]
    path = f'./data/earthquakes/{start}/{url}.json'
    with open(path, 'w') as fi:
        json.dump(properties, fi)

## Exercise (individual) - Wikipedia API

Now for an open-ended exercise for you! Your task is to:
- create a database of countries
- in a folder called `countries` (you will need to make the folder - you can do this in bash or Python)
- each country in it's own folder
- start with germany & new zealand

V1 of your program should:
- save the url you use to request the data
- save the title
- save the `line` parameter of each section (`data['parse']['sections']`)
- save all in a single JSON

V2 of your program should also:
- save all '.png' & '.jpg' images as images, with the url as the image name
- save all external links as CSV

Much of the work will be understanding how the Wikipedia API works - useful resources are below:
- [Main API page](https://www.mediawiki.org/wiki/API:Main_page)
- [What the actions are](https://www.mediawiki.org/w/api.php)
- [Python examples](https://github.com/wikimedia/mediawiki-api-demos/tree/master/python)

Please also feel free to work on another API - happy to assist you with this as well :)

In [71]:

url = f"https://en.wikipedia.org/w/api.php?action=parse&page={term}&format=json"
res = requests.get(url)
data = res.json()

data['parse']['sections']

[{'toclevel': 1,
  'level': '2',
  'line': 'Etymology',
  'number': '1',
  'index': '1',
  'fromtitle': 'Germany',
  'byteoffset': 13511,
  'anchor': 'Etymology'},
 {'toclevel': 1,
  'level': '2',
  'line': 'History',
  'number': '2',
  'index': '2',
  'fromtitle': 'Germany',
  'byteoffset': 16045,
  'anchor': 'History'},
 {'toclevel': 2,
  'level': '3',
  'line': 'Germanic tribes and Frankish Empire',
  'number': '2.1',
  'index': '3',
  'fromtitle': 'Germany',
  'byteoffset': 19271,
  'anchor': 'Germanic_tribes_and_Frankish_Empire'},
 {'toclevel': 2,
  'level': '3',
  'line': 'East Francia and Holy Roman Empire',
  'number': '2.2',
  'index': '4',
  'fromtitle': 'Germany',
  'byteoffset': 23116,
  'anchor': 'East_Francia_and_Holy_Roman_Empire'},
 {'toclevel': 2,
  'level': '3',
  'line': 'German Confederation and Empire',
  'number': '2.3',
  'index': '5',
  'fromtitle': 'Germany',
  'byteoffset': 30746,
  'anchor': 'German_Confederation_and_Empire'},
 {'toclevel': 2,
  'level': '3',

In [ ]:
import json
import requests

term = 'germany'
os.makedirs(f'./data/countries/{term}', exist_ok=True)
os.makedirs(f'./data/countries/{term}/images', exist_ok=True)

url = f"https://en.wikipedia.org/w/api.php?action=parse&page={term}&format=json"
res = requests.get(url)
data = res.json()

data = data['parse']
out = {
    'url': url,
    'title': data['title']
}

path = os.path.join('data', 'countries', term, 'data.json')
with open(path, 'w') as fi:
    json.dump(out, fi)
    
for img in data['images']:
    url = f'https://en.wikipedia.org/w/api.php?action=query&format=json&list=allimages&aifrom={img}&ailimit=1'
    res = requests.get(url)
    res = res.json()
    with open(f'./data/countries/{term}/images/{img}', 'wb') as fi:
        url = res['query']['allimages'][0]['url']
        res = requests.get(url)
        fi.write(res.content)

In [67]:
import json
import requests

term = 'germany'
os.makedirs(f'./data/countries/{term}', exist_ok=True)
os.makedirs(f'./data/countries/{term}/images', exist_ok=True)

url = f"https://en.wikipedia.org/w/api.php?action=parse&page={term}&format=json"
res = requests.get(url)
data = res.json()

data = data['parse']
out = {
    'url': url,
    'title': data['title']
}

path = os.path.join('data', 'countries', term, 'data.json')
with open(path, 'w') as fi:
    json.dump(out, fi)
    
for img in data['images']:
    url = f'https://en.wikipedia.org/w/api.php?action=query&format=json&list=allimages&aifrom={img}&ailimit=1'
    res = requests.get(url)
    res = res.json()
    with open(f'./data/countries/{term}/images/{img}', 'wb') as fi:
        url = res['query']['allimages'][0]['url']
        res = requests.get(url)
        fi.write(res.content)